In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import utility_functions
%matplotlib inline
plt.style.use('ggplot')
pd.set_option('display.max_columns', 200) # So we can see all columns
pd.set_option('display.max_rows',200)

In [2]:
import importlib
importlib.reload(utility_functions)

<module 'utility_functions' from 'd:\\Banque Misr Internship\\loan-defaulter-prediction\\Final Notebooks\\utility_functions.py'>

In [3]:
df = pd.read_csv('D:/Banque Misr Internship/Loan Datasets/sampled_data_1.csv')

In [4]:
df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].abs()
df['DAYS_BIRTH'] = df['DAYS_BIRTH'].abs()

In [5]:
df_cleaned = df.replace('XNA', pd.NA)

In [6]:
categorical_features = []
categorical_features = utility_functions.get_categorical_features(df_cleaned)
print("Categorical Features:",categorical_features )

Categorical Features: ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'ORGANIZATION_TYPE']


In [7]:

# One-hot encode the categorical columns
df_encoded = pd.get_dummies(df_cleaned, columns=categorical_features, drop_first=False)

# Check the resulting DataFrame
df_encoded.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_YEAR,CODE_GENDER_F,CODE_GENDER_M,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Culture,ORGANIZATION_TYPE_Electricity,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Hotel,ORGANIZATION_TYPE_Housing,ORGANIZATION_TYPE_Industry: type 1,ORGANIZATION_TYPE_Industry: type 10,ORGANIZATION_TYPE_Industry: type 11,ORGANIZATION_TYPE_Industry: type 12,ORGANIZATION_TYPE_Industry: type 13,ORGANIZATION_TYPE_Industry: type 2,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Industry: type 4,ORGANIZATION_TYPE_Industry: type 5,ORGANIZATION_TYPE_Industry: type 6,ORGANIZATION_TYPE_Industry: type 7,ORGANIZATION_TYPE_Industry: type 8,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Legal Services,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,9461,637,0.262949,0.139376,2.0,2.0,2.0,2.0,-1134.0,1,0.0,1.0,False,True,True,False,False,True,False,False,False,False,False,False,True,False,False,False,

In [9]:
nulls = utility_functions.get_numerical_summary(df_encoded)

AMT_ANNUITY : 12 (0.004%)
AMT_GOODS_PRICE : 278 (0.09%)
EXT_SOURCE_2 : 660 (0.215%)
EXT_SOURCE_3 : 60965 (19.825%)
OBS_30_CNT_SOCIAL_CIRCLE : 1021 (0.332%)
DEF_30_CNT_SOCIAL_CIRCLE : 1021 (0.332%)
OBS_60_CNT_SOCIAL_CIRCLE : 1021 (0.332%)
DEF_60_CNT_SOCIAL_CIRCLE : 1021 (0.332%)
DAYS_LAST_PHONE_CHANGE : 1 (0.0%)
AMT_REQ_CREDIT_BUREAU_HOUR : 41519 (13.502%)
AMT_REQ_CREDIT_BUREAU_YEAR : 41519 (13.502%)


In [10]:
from sklearn.impute import SimpleImputer

# Create a copy of the original dataframe to work on
df = df_encoded.copy()

# Impute numerical columns
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
for col in numerical_cols:
    if df[col].isnull().sum() > 0:
        imputer = SimpleImputer(strategy='mean')  
        df[col] = imputer.fit_transform(df[[col]])

# Impute categorical columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
for col in categorical_cols:
    if df[col].isnull().sum() > 0:
        imputer = SimpleImputer(strategy='most_frequent')  
        df[col] = imputer.fit_transform(df[[col]])

# Check if there are any remaining nulls
print(df.isnull().sum().sort_values(ascending=False))


SK_ID_CURR                                           0
ORGANIZATION_TYPE_Culture                            0
ORGANIZATION_TYPE_Industry: type 5                   0
ORGANIZATION_TYPE_Industry: type 4                   0
ORGANIZATION_TYPE_Industry: type 3                   0
ORGANIZATION_TYPE_Industry: type 2                   0
ORGANIZATION_TYPE_Industry: type 13                  0
ORGANIZATION_TYPE_Industry: type 12                  0
ORGANIZATION_TYPE_Industry: type 11                  0
ORGANIZATION_TYPE_Industry: type 10                  0
ORGANIZATION_TYPE_Industry: type 1                   0
ORGANIZATION_TYPE_Housing                            0
ORGANIZATION_TYPE_Hotel                              0
ORGANIZATION_TYPE_Government                         0
ORGANIZATION_TYPE_Emergency                          0
ORGANIZATION_TYPE_Electricity                        0
ORGANIZATION_TYPE_Construction                       0
OCCUPATION_TYPE_Low-skill Laborers                   0
ORGANIZATI

In [ ]:
final_df = df_encoded.dropna()


In [11]:
filepath = Path('D:/Banque Misr Internship/Loan Datasets/imputed_dataset.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)